In [65]:
from deap import base, creator, tools
import random
import itertools
import string

# define the signal and meaning spaces
signals = list(string.ascii_lowercase)  # All possible signals
#meanings = [0, 1, 2, 3]         # All possible meanings
meanings = list(range(0,26))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax, local_state=None)

toolbox = base.Toolbox()

toolbox.register("attr_signal", random.choice, signals)
toolbox.register("attr_meaning", random.choice, meanings)
toolbox.register("attr_local_state", random.randint, 0, 3)
toolbox.register("attr_global_state", random.randint, 0, 3)

GLOBAL = toolbox.attr_global_state()
print("Global State: ", GLOBAL)

def create_individual():
    trans_genes = [toolbox.attr_signal() for _ in meanings]  
    recept_genes = [toolbox.attr_meaning() for _ in signals]  
    individual = creator.Individual(trans_genes + recept_genes)
    individual.local_state = toolbox.attr_local_state() 
    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


#GLOBAL_STATE = random.randint(0, 3)
GlOBAL_STATE = random.choice(meanings)

def pairwise_communication(speaker, listener):

    # speaker encodes signals
    local_meaning = speaker.local_state
    global_meaning = GLOBAL

    semantic_local = speaker[local_meaning]
    semantic_global = speaker[global_meaning]

    # listener decodes local signal
    local_index = 0
    for signal in signals:
        if semantic_local == signal:
            decoded_local = listener[local_index + len(meanings)]
        local_index += 1


    # if semantic_local == 'a':
    #     decoded_local = listener[4]
    # elif semantic_local == 'b':
    #     decoded_local = listener[5]
    # elif semantic_local == 'c':
    #     decoded_local = listener[6]
    # else:
    #     decoded_local = listener[7]

    # listener decodes global signal
    global_index = 0
    for signal in signals:
        if semantic_global == signal:
            decoded_global = listener[global_index + len(meanings)]
        global_index += 1




    # if semantic_global == 'a':
    #     decoded_global = listener[4]
    # elif semantic_global == 'b':
    #     decoded_global = listener[5]
    # elif semantic_global == 'c':
    #     decoded_global = listener[6]
    # else:
    #     decoded_global = listener[7]

    fitness = 0
    if local_meaning == decoded_local:
        fitness += 1
    if global_meaning == decoded_global:
        fitness += 1
    return fitness   

# evaluate group
def evaluate_group(group):
    for ind in group:
        ind.fitness.values = (0,)  # reset fitness
    
    for speaker, listener in itertools.permutations(group, 2):
        fitness_bonus = pairwise_communication(speaker, listener)
        speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

# create pop
population = toolbox.population(n=100)

GROUP_SIZE = 10

# eval groups
for i in range(0, len(population), GROUP_SIZE):
    group = population[i:i + GROUP_SIZE]
    evaluate_group(group)

for ind in population:
    print(ind, ind.fitness.values)


Global State:  3
['x', 's', 't', 'j', 'l', 'a', 'q', 'n', 'f', 'i', 'x', 'a', 'h', 'j', 'i', 'w', 'v', 'k', 'z', 'f', 'h', 'h', 'a', 'g', 't', 'd', 18, 22, 21, 4, 7, 6, 1, 1, 21, 23, 12, 13, 6, 20, 17, 11, 25, 25, 10, 4, 5, 11, 8, 24, 17, 4] (0.0,)
['e', 'y', 'a', 'l', 'm', 'j', 'a', 'r', 'w', 'h', 'u', 'i', 'y', 'p', 'k', 'z', 'j', 'z', 'q', 'j', 'p', 'i', 'b', 'c', 'm', 'c', 19, 10, 10, 8, 16, 18, 20, 11, 23, 11, 12, 0, 11, 19, 16, 11, 8, 17, 16, 1, 22, 9, 24, 0, 11, 23] (1.0,)
['l', 'n', 'm', 'o', 'e', 'w', 'k', 'z', 'd', 'k', 'm', 'b', 'q', 'm', 'm', 'e', 'y', 'r', 's', 'f', 'i', 'g', 'q', 'f', 'g', 'n', 10, 9, 2, 25, 13, 16, 15, 24, 3, 8, 10, 25, 12, 25, 10, 25, 7, 21, 18, 20, 16, 8, 21, 15, 18, 18] (1.0,)
['l', 'l', 'w', 'n', 't', 'd', 'l', 'u', 'w', 'n', 'm', 'z', 'q', 'l', 'j', 'k', 'h', 'n', 's', 'b', 'a', 'd', 'v', 'p', 'r', 'w', 13, 16, 21, 15, 16, 2, 5, 10, 2, 21, 11, 25, 11, 4, 20, 8, 16, 25, 20, 10, 10, 0, 23, 17, 20, 16] (0.0,)
['s', 'a', 'n', 'l', 'o', 'k', 'w', 'g', 't

In [66]:
from deap import algorithms
import numpy

# Parameters
NUM_GENERATIONS = 100
CXPB = 0.5  # Crossover probability
MUTPB = 0.0  # Mutation probability (seems better at 0 for now)

# mut for signals
def mutateInd(individual, indpb):
    for i in range(len(meanings)):
        if random.random() < indpb:
            individual[i] = toolbox.attr_signal()
    for i in range(len(signals)):
        if random.random() < indpb:
            individual[i + len(meanings)] = toolbox.attr_meaning()
    return individual,


toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", mutateInd, indpb=0.1)
toolbox.register("select", tools.selRoulette) # inspired by paper
toolbox.register("evaluate", evaluate_group)


stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

# ea
for gen in range(NUM_GENERATIONS):
    #GLOBAL_STATE = random.randint(0,3)
    GlOBAL_STATE = random.choice(meanings)

    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # mutation and crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # eval for invalid individuals
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    for group in [invalid_ind[i:i + GROUP_SIZE] for i in range(0, len(invalid_ind), GROUP_SIZE)]:
        evaluate_group(group)

    
    population[:] = offspring

    fits = [ind.fitness.values[0] for ind in population]

    length = len(population)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"Generation {gen}: Min {min(fits)}, Max {max(fits)}, Avg {mean}, Std {std}")

# print final pop
for ind in population:
    print(ind, ind.fitness.values)


Generation 0: Min 0.0, Max 4.0, Avg 1.98, Std 1.0952625256074455
Generation 1: Min 0.0, Max 6.0, Avg 2.07, Std 1.3583445807305303
Generation 2: Min 0.0, Max 10.0, Avg 2.23, Std 1.719622051498526
Generation 3: Min 0.0, Max 10.0, Avg 3.11, Std 2.222138609538118
Generation 4: Min 0.0, Max 10.0, Avg 3.67, Std 2.53793222919762
Generation 5: Min 0.0, Max 16.0, Avg 4.73, Std 3.5181102882087134
Generation 6: Min 0.0, Max 15.0, Avg 7.0, Std 3.6687872655688283
Generation 7: Min 2.0, Max 22.0, Avg 10.41, Std 4.6884858963209
Generation 8: Min 2.0, Max 24.0, Avg 13.99, Std 4.9162892510510385
Generation 9: Min 0.0, Max 24.0, Avg 14.74, Std 4.6618022266072145
Generation 10: Min 5.0, Max 30.0, Avg 18.66, Std 5.772728990694087
Generation 11: Min 8.0, Max 32.0, Avg 19.61, Std 6.05622819913517
Generation 12: Min 2.0, Max 33.0, Avg 22.1, Std 5.682429058070146
Generation 13: Min 10.0, Max 35.0, Avg 23.47, Std 5.755788390828846
Generation 14: Min 2.0, Max 33.0, Avg 24.27, Std 5.29689531707019
Generation 15:

In [67]:
from collections import Counter
import statistics
from statistics import mean as calc_mean

# check signal and meaning modes
def calculate_mode_with_percentage(population):

    transposed_population = list(zip(*population))

    modes_with_percentages = []
    population_size = len(population)
    for gene_index in transposed_population:
        # count values
        counter = Counter(gene_index)
        mode_value, mode_count = counter.most_common(1)[0]
        mode_percentage = (mode_count / population_size) * 100
        modes_with_percentages.append((mode_value, mode_percentage))

    return modes_with_percentages

modes_with_percentages = calculate_mode_with_percentage(population)
print("Modes and percentages for each index in the final population:")
for index, (mode, percentage) in enumerate(modes_with_percentages):
    print(f"Index {index}: {mode} ({percentage:.2f}%)")

signal_modes_with_percentages = calculate_mode_with_percentage([ind[:len(meanings)] for ind in population])
meaning_modes_with_percentages = calculate_mode_with_percentage([ind[len(meanings):] for ind in population])

all_meaning_percents = []
all_signal_percents = []

print("\nModes and percentages for signal indices in the final population:")
for index, (mode, percentage) in enumerate(signal_modes_with_percentages):
    all_signal_percents.append(percentage)
    print(f"Signal Index {index}: {mode} ({percentage:.2f}%)")

print("\nModes and percentages for meaning indices in the final population:")
for index, (mode, percentage) in enumerate(meaning_modes_with_percentages):
    all_meaning_percents.append(percentage)
    print(f"Meaning Index {index}: {mode} ({percentage:.2f}%)")

average_signal_percentage = calc_mean(all_signal_percents)
average_meaning_percentage = calc_mean(all_meaning_percents)

print("Average Signal Percentage: ", average_signal_percentage)
print("Average Meaning Percentage: ", average_meaning_percentage)



Modes and percentages for each index in the final population:
Index 0: q (98.00%)
Index 1: t (80.00%)
Index 2: d (88.00%)
Index 3: h (100.00%)
Index 4: x (100.00%)
Index 5: v (71.00%)
Index 6: j (53.00%)
Index 7: d (71.00%)
Index 8: q (78.00%)
Index 9: g (100.00%)
Index 10: p (92.00%)
Index 11: u (46.00%)
Index 12: w (100.00%)
Index 13: d (82.00%)
Index 14: u (70.00%)
Index 15: d (74.00%)
Index 16: i (50.00%)
Index 17: c (82.00%)
Index 18: g (88.00%)
Index 19: a (100.00%)
Index 20: z (92.00%)
Index 21: x (66.00%)
Index 22: j (100.00%)
Index 23: s (66.00%)
Index 24: f (65.00%)
Index 25: i (100.00%)
Index 26: 2 (100.00%)
Index 27: 14 (52.00%)
Index 28: 5 (100.00%)
Index 29: 0 (63.00%)
Index 30: 4 (76.00%)
Index 31: 11 (100.00%)
Index 32: 11 (100.00%)
Index 33: 3 (100.00%)
Index 34: 0 (47.00%)
Index 35: 4 (68.00%)
Index 36: 20 (66.00%)
Index 37: 11 (52.00%)
Index 38: 11 (54.00%)
Index 39: 17 (70.00%)
Index 40: 14 (62.00%)
Index 41: 10 (100.00%)
Index 42: 24 (57.00%)
Index 43: 18 (100.00%)